In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sentence_transformers import SentenceTransformer

d:\proga\computer_technology_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("D:\downloads_d\movies.csv")
df

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\mikha\AppData\Local\Temp\ipykernel_6320\3900269106.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv("D:\downloads_d\movies.csv")


,Unnamed: 0,id,title,overview,release_date,popularity,vote_average,vote_count
0,0,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",1995-10-20,18.433,8.7,2763
1,1,724089,Gabriel's Inferno Part II,Professor Gabriel Emerson finally learns the t...,2020-07-31,8.439,8.7,1223
2,2,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,1994-09-23,65.570,8.7,18637
3,3,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,63.277,8.7,14052
4,4,761053,Gabriel's Inferno Part III,The final part of the film adaption of the ero...,2020-11-19,26.691,8.7,773
...,...,...,...,...,...,...,...,...
8546,8546,13805,Disaster Movie,"The filmmaking team behind the hits ""Scary Mov...",2008-08-29,14.630,3.2,714
8547,8547,5491,Battlefield Earth,"In the year 3000, man is no match for the Psyc...",2000-05-12,10.647,3.0,543
8548,8548,14164,Dragonball Evolution,The young warrior Son Goku sets out on a quest...,2009-03-12,32.244,2.8,1447
8549,8549,11059,House of the Dead,"Set on an island off the coast, a techno rave ...",2003-04-11,14.502,2.8,238


заполнение пустых значений 

In [5]:
df = df.dropna(subset=['title', 'overview'])
df.isna().sum()

Unnamed: 0      0
id              0
title           0
overview        0
release_date    0
popularity      0
vote_average    0
vote_count      0
dtype: int64

создаем текстовые эмбединги 

In [6]:
texts = (df["title"] + " " + df["overview"]).tolist()
model = SentenceTransformer("all-MiniLM-L6-v2")
text_embeddings = model.encode(texts, show_progress_bar=True)

d:\proga\computer_technology_project\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mikha\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Fallin

нормализуем числовые признаки

In [7]:
numeric_features = df[["popularity", "vote_average", "vote_count"]]
scaler = MinMaxScaler()
numeric_embeddings = scaler.fit_transform(numeric_features)

объединяем все в один эмбединг 

In [8]:
final_embeddings = np.hstack([text_embeddings, numeric_embeddings])

print("Финалные эмбединги:", final_embeddings.shape)

Финалные эмбединги: (8531, 387)


In [21]:
# Предположим, у нас уже есть:
# - df (исходный DataFrame с id, title)
# - final_embeddings (массив эмбедингов, shape = [n_movies, embedding_dim])

# Преобразуем эмбеддинги в строки
embedding_strings = [json.dumps(embedding.tolist()) for embedding in final_embeddings]

# Создаём новый DataFrame
result_df = pd.DataFrame({
    "id": df["id"],
    "title": df["title"],
    "popularity": df["popularity"],
    "embedding": embedding_strings  # сохраняем эмбеддинги как строки
})



In [23]:
result_df['embedding']

0       [-0.06061665341258049, 0.06988733261823654, 0....
1       [-0.035317450761795044, -0.0042189364321529865...
2       [-0.03228230029344559, -0.04405484348535538, -...
3       [-0.08061544597148895, 0.03567180782556534, -0...
4       [-0.03265352174639702, -0.023313350975513458, ...
                              ...                        
8546    [-0.053737789392471313, -0.04901444911956787, ...
8547    [-0.08673122525215149, 0.10429753363132477, -0...
8548    [-0.059643641114234924, 0.0857674777507782, -0...
8549    [-0.03159128502011299, 0.000796342792455107, -...
8550    [0.051322948187589645, 0.004296405706554651, -...
Name: embedding, Length: 8531, dtype: object

In [25]:
import psycopg2
import numpy as np

# Подключение к базе данных PostgreSQL
conn = psycopg2.connect(database="postgres", user="postgres", password="misha", host="", port="")
cursor = conn.cursor()

# Перебираем строки DataFrame и вставляем их в таблицу movies
for idx, row in result_df.iterrows():
    embedding = row["embedding"]
    # Преобразуем эмбеддинг в строку в формате массива для PostgreSQL
    embedding_str = "{" + ",".join(map(str, embedding)) + "}"  # преобразуем в строку для PostgreSQL
    cursor.execute("""
        INSERT INTO movies (id, title, popularity, embedding)
        VALUES (%s, %s, %s, %s)
    """, (row["id"], row["title"], row["popularity"], embedding_str))

# Подтверждаем изменения в базе данных
conn.commit()

# Закрываем соединение
cursor.close()
conn.close()

print("Данные успешно загружены в таблицу 'movies' в PostgreSQL!")


InvalidTextRepresentation: invalid input syntax for type double precision: "["
LINE 3: ...ES (19404, 'Dilwale Dulhania Le Jayenge', 18.433, '{[,-,0,.,...
                                                             ^
